 #  Import et Nettoyage des données Projet Immo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model
import pickle
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


Import du csv

In [ ]:
immo_df = pd.read_csv(".csv", delimiter=",")
immo_df

In [ ]:
immo_df.rename(columns = {'Unnamed: 0':'index'}, inplace = True)
immo_df

encodage ocean proximity

In [ ]:
enc = OneHotEncoder(sparse=False).fit(immo_df[["ocean_proximity"]])
encoded = enc.transform(immo_df[["ocean_proximity"]])
encoded_df = pd.DataFrame(encoded, columns=enc.get_feature_names_out())
immo_df2 = immo_df.join(encoded_df)
immo_df2

In [ ]:
immo_df2=immo_df2.drop("ocean_proximity", axis=1)

Gestion des valeurs manquante

In [ ]:

if immo_df2.isnull().values.any()== True:

    # Imputing with MICE
    df_mice = immo_df2.filter(['index','longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','ocean_proximity_<1H OCEAN','ocean_proximity_INLAND','ocean_proximity_ISLAND','ocean_proximity_NEAR BAY','ocean_proximity_NEAR OCEAN','median_house_value'], axis=1).copy()

    # Define MICE Imputer and fill missing values
    mice_imputer = IterativeImputer(estimator=linear_model.BayesianRidge(), n_nearest_features=None, imputation_order='ascending')

    df_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(df_mice), columns=df_mice.columns)
    
df3 = df_mice_imputed.copy()


"Gestion des outliers"

In [ ]:
immo_df3=df3.loc[df3['population']<28565.0]

Creation de la variable room_per_households

In [ ]:
immo_df4 = immo_df3.copy()
immo_df4['room_per_households']= immo_df4['total_rooms']/immo_df4['households']

In [ ]:
immo_df4=immo_df4.drop("total_rooms", axis=1)

In [ ]:
immo_df4=immo_df4.drop("households", axis=1)

scale avec le log

In [ ]:
immo_df5=immo_df4.copy()
liste = ['total_bedrooms','population','room_per_households','median_income', 'median_house_value']
for index, i in enumerate(liste):
    immo_df5[F'{i}_log'] = np.log(np.absolute(immo_df5[i]))
    immo_df5=immo_df5.drop(i, axis=1)

In [ ]:
y = immo_df5["median_house_value_log"]
X = immo_df5.copy()

In [ ]:
X= X.drop("median_house_value_log", axis=1)
X


In [ ]:
pickled_model = pickle.load(open('linear_regression.pkl', 'rb'))
pickled_model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

scores = cross_validate(pickled_model, X, y, cv=5)
predictions = pickled_model.predict(X)
predictions_original_scale = np.exp(predictions)
y_original_scale = np.exp(y)



In [ ]:
average_r2_real=scores['test_score'].mean()
rmse_real = np.sqrt(mean_squared_error(y_original_scale, predictions_original_scale))
mae_real = mean_absolute_error(y_original_scale, predictions_original_scale)

print("The model performance for testing set")
print("--------------------------------------")
print('Average MAE:', mae_real)
print(' Average RMSE: ', rmse_real)
print("Average R2: ", average_r2_real)